First, let's import the needed libraries.

In [ ]:
!pip install selenium 
!apt-get update 
!apt install chromium-chromedriver

In [ ]:
!sudo apt-get install unixodbc-dev
!pip install pyodbc

In [ ]:
from bs4 import BeautifulSoup
import requests
import html5lib
import pandas as pd
import time
import datetime
import pyodbc

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
%%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

Now lets define which stock tickers we will retrieve:

In [ ]:
tickers = [ "VALE3.SA", "TAEE11.SA","BBSE3.SA", "IVVB11.SA", "BBAS3.SA", "PETR4.SA", "MGLU3.SA","VIIA3.SA", "WEGE3.SA", "TRPL4.SA",
           "FLRY3.SA","CPLE6.SA", "ITSA4.SA", "PETZ3.SA","ITUB4.SA", "BBDC4.SA", "SAPR4.SA", "KLBN4.SA", "UNIP6.SA", 
           "GGBR4.SA", "TUPY3.SA","AGRO3.SA", "CMIG4.SA", "OIBR4.SA" ]

In [ ]:
class Stock:
  def __init__(self, id, quotations):
    self.id = id
    self.quotations = quotations 

And, for each ticker, scrape its data from Yahoo! Finance website:

In [ ]:
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=personal-sql-server.database.windows.net;'
                      'Database=stocks-bd;'
                      'UID=gabriel;'
                      'PWD=Pass1234;')

cursor = conn.cursor()

stocks = []

flag = False

for ticker in tickers:
  
  base_url = 'https://finance.yahoo.com/quote/'
  url_suffix = ticker + '/history?period1=1510358400&period2=1668124800&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true'
  ticker_url = base_url + url_suffix
  headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246", 'COOKIES_ENABLED': True}

  browser.get(ticker_url)
  # print(browser.page_source)
  if(flag == False):
    button = browser.find_element(By.NAME, 'agree')
    button.click()
    flag = True

  print(browser.current_url)
  elem = browser.find_element(By.TAG_NAME,'body')
  no_of_pagedowns = 200

  while no_of_pagedowns:
      elem.send_keys(Keys.PAGE_DOWN)
      time.sleep(0.5)
      no_of_pagedowns-=1

  # r = requests.get(url=ticker_url, headers=headers)
  # soup = BeautifulSoup(r.content, 'html5lib')

  soup = BeautifulSoup(browser.page_source, 'html5lib')
  company_name = soup.find( 'h1', attrs = {'class': 'D(ib) Fz(18px)'} ).text
  company_name_clean = company_name.split('(')[0]
  print('Company name: ' + company_name_clean)

  cursor.execute("select StockId from Stock where CompanyTicker = '" + ticker + "'")
  # if(cursor.fetchone()):
  #   stock_id = cursor.fetchone()[0]
  #   print("tem")
  # else:
  #   cursor.execute("insert into Stock (CompanyName, CompanyTicker) values ('" + company_name_clean + "', '" + ticker + "')")
  #   stock_id = cursor.execute('SELECT SCOPE_IDENTITY()').fetchone()[0]
  #   print("n tem")
  # print(stock_id)

  quotations = []

  table = soup.find( 'table', attrs = {'data-test':'historical-prices'} )
  for row in table.findAll( 'tr', attrs = {'class': 'BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)'} ) :
    data = row.findAll( 'td')
    if(len(data) > 4):
      date_str = data[0].find('span').text
      value = data[4].find('span').text
      # print('Date: ' + date_str + '  Closing value: ' + value)
      quotations.append((date_str, value))
  
  stocks.append(stock)
  stock = Stock(stock_id, quotations)
  print(quotations)


for stock in stocks:
  query = 'INSERT INTO StockQuotation (StockId, QuotationDate, QuotationValue) VALUES '
  for quotation in stock.quotations:
    query += "(" + str(stock.id) + ", '" + datetime.datetime.strptime(quotation[0], '%b %d, %Y').strftime('%Y-%m-%d') + "', " + str(quotation[1]) + "), "
  print(query[:-2])
  cursor.execute(query[:-2])
conn.commit()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


https://finance.yahoo.com/quote/VALE3.SA/history?period1=1510358400&period2=1668124800&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true&guccounter=1
Company name: Vale S.A. 
[('Nov 10, 2022', '74.55'), ('Nov 09, 2022', '73.15'), ('Nov 08, 2022', '74.05'), ('Nov 07, 2022', '71.87'), ('Nov 04, 2022', '72.25'), ('Nov 03, 2022', '67.15'), ('Nov 01, 2022', '69.17'), ('Oct 31, 2022', '67.13'), ('Oct 28, 2022', '67.45'), ('Oct 27, 2022', '70.91'), ('Oct 26, 2022', '73.53'), ('Oct 25, 2022', '71.93'), ('Oct 24, 2022', '71.93'), ('Oct 21, 2022', '74.13'), ('Oct 20, 2022', '72.02'), ('Oct 19, 2022', '71.11'), ('Oct 18, 2022', '71.96'), ('Oct 17, 2022', '70.93'), ('Oct 14, 2022', '69.83'), ('Oct 13, 2022', '72.18'), ('Oct 11, 2022', '73.49'), ('Oct 10, 2022', '73.99'), ('Oct 07, 2022', '75.51'), ('Oct 06, 2022', '75.23'), ('Oct 05, 2022', '76.96'), ('Oct 04, 2022', '75.79'), ('Oct 03, 2022', '74.00'), ('Sep 30, 2022', '72.04'), ('Sep 29, 2022', '68.43'), ('Sep 28, 2022', '68.34')

KeyboardInterrupt: ignored